In [8]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.pardir)))

In [9]:
from src.data_loader import load_s1_mat

file_path = '../data/raw/S1.mat'
eeg_trials, attended_labels, fs = load_s1_mat(file_path)

print(f"Number of trials: {len(eeg_trials)}")
print(f"Sampling rate: {fs} Hz")
print(f"Shape of first trial EEG: {eeg_trials[0].shape} (samples × channels)")
print(f"First 5 attended labels: {attended_labels[:5]}")


Number of trials: 20
Sampling rate: 128 Hz
Shape of first trial EEG: (49792, 64) (samples × channels)
First 5 attended labels: ['R', 'L', 'R', 'L', 'R']


In [10]:
from src.preprocess import preprocess_eeg_trials
from src.data_loader import load_s1_mat

file_path = '../data/raw/S1.mat'
eeg_trials, attended_labels, fs = load_s1_mat(file_path)

print(f"Original sampling rate: {fs} Hz")
print(f"Number of trials before preprocessing: {len(eeg_trials)}")

# Example preprocessing: filter 1-40 Hz, downsample to 128 Hz
processed_trials, new_fs = preprocess_eeg_trials(eeg_trials, sfreq=fs, l_freq=1, h_freq=40, downsample=128)

print(f"New sampling rate after downsampling: {new_fs} Hz")
print(f"Shape of first processed trial: {processed_trials[0].shape} (channels x samples)")


Original sampling rate: 128 Hz
Number of trials before preprocessing: 20
Creating RawArray with float64 data, n_channels=64, n_times=49792
    Range : 0 ... 49791 =      0.000 ...   388.992 secs
Ready.
Creating RawArray with float64 data, n_channels=64, n_times=50048
    Range : 0 ... 50047 =      0.000 ...   390.992 secs
Ready.
Creating RawArray with float64 data, n_channels=64, n_times=49792
    Range : 0 ... 49791 =      0.000 ...   388.992 secs
Ready.
Creating RawArray with float64 data, n_channels=64, n_times=51072
    Range : 0 ... 51071 =      0.000 ...   398.992 secs
Ready.
Creating RawArray with float64 data, n_channels=64, n_times=49792
    Range : 0 ... 49791 =      0.000 ...   388.992 secs
Ready.
Creating RawArray with float64 data, n_channels=64, n_times=50048
    Range : 0 ... 50047 =      0.000 ...   390.992 secs
Ready.
Creating RawArray with float64 data, n_channels=64, n_times=49792
    Range : 0 ... 49791 =      0.000 ...   388.992 secs
Ready.
Creating RawArray with f

In [11]:
from src.dataset import segment_trials, EEGAttentionDataset
from torch.utils.data import DataLoader

# Segment trials into fixed-length windows (e.g., 2 sec = 256 samples at 128Hz)
segment_length = 256
segmented_trials = segment_trials(processed_trials, segment_length=segment_length, overlap=0)

# Expand labels for each segment
segmented_labels = []
for label, trial in zip(attended_labels, processed_trials):
    n_segments = (trial.shape[1] - segment_length) // segment_length + 1
    segmented_labels.extend([label] * n_segments)

# Create Dataset and DataLoader
dataset = EEGAttentionDataset(segmented_trials, segmented_labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Check batch shapes
for batch_eeg, batch_labels in dataloader:
    print(batch_eeg.shape)  # (batch_size, channels, samples)
    print(batch_labels)
    break

ImportError: cannot import name 'segment_trials' from 'src.dataset' (c:\Users\Ruchita.Abhang\PycharmProjects\LLM Project New\AAD-LLM\src\dataset.py)